In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import requests
import json
from tqdm import tqdm
import time
from datetime import datetime
import os
import glob
import pickle
from typing import Dict, List, Optional, Tuple


In [ ]:
# Configuración de API
OPENROUTER_API_KEY = "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"  # Obtener de https://openrouter.ai/keys
OPENROUTER_API_URL = "https://openrouter.ai/api/v1/chat/completions"

# Modelo recomendado (puedes cambiar según necesidades y presupuesto)
MODEL = "google/gemini-2.5-flash"  # Rápido y económico
# Alternativas: "openai/gpt-3.5-turbo", "meta-llama/llama-3-8b-instruct"

# Headers para las peticiones
headers = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json",
    "HTTP-Referer": "http://localhost:8888",  # Requerido por OpenRouter
    "X-Title": "TrustPilot Analysis"
}

In [ ]:
# Cargar el CSV con las reseñas
df = pd.read_csv('', encoding='utf-8-sig')

# Verificar columnas vacías
columnas_analizar = ['language', 'sentiment', 'emotion', 'customer_gender', 
                     'main_topic', 'keywords', 'tourist_type', 'group_type']

print(f"Total de reseñas: {len(df)}")
print(f"Reseñas con texto: {df['review_text'].notna().sum()}")

In [ ]:
def crear_prompt_analisis(review_text, customer_name):
    """Crea el prompt para analizar una reseña"""
    
    prompt = f"""
Eres un analizador especializado en evaluación de reseñas turísticas y análisis de sentimientos. Para cada texto que recibas, deberás analizar y proporcionar la siguiente información separada por el delimitador "|":

RESEÑA A ANALIZAR:
Texto: {review_text}
Cliente: {customer_name}

ANÁLISIS REQUERIDO (responde cada campo separado por "|"):

1. Sentiment: Clasifica como "Positivo", "Negativo" o "Neutro"
2. Sentiment_score: Evalúa en escala de -1 a +1 (-1=extremadamente negativo, 0=neutro, +1=extremadamente positivo)
3. Emotion: Identifica una emoción (joy, surprise, neutral, sadness, disgust, anger, fear)
4. Emotion_intensity: Intensidad de 1-5 (1=muy leve, 5=muy intensa)
5. Customer_gender: Basado en el nombre (masculino, femenino, unknown)
6. Topic: Tema principal (Atención al cliente, Limpieza, Instalaciones, Relación calidad-precio, Servicios, Ubicación, Ética y sostenibilidad, Check-in y Check-out, Comodidad y descanso, Oferta gastronómica, Facilidad de reserva y accesibilidad digital, Animación y actividades, Seguridad)
7. Keywords: 3-5 términos relevantes separados por comas SIN espacios
8. Customer_type: Promotor, Leal, Neutral, Crítico, Oportunista
9. Tourist_type: Turista de ocio, cultural, naturaleza, aventura, compras, espiritual/religioso, gastronómico, deportivo, wellness, solidario/voluntario
10. Group_type: familiar, amigos, pareja, solitario, grupo organizado

FORMATO DE RESPUESTA:
Responde ÚNICAMENTE con los valores separados por "|" en el orden exacto listado arriba.
Si no puedes determinar algún campo, usa "unknown".
NO incluyas espacios antes o después de los pipes.

Ejemplo: Positivo|0.8|joy|4|femenino|Atención al cliente|excelente,servicio,amable|Promotor|Turista de ocio|pareja
"""
    return prompt

In [ ]:
def analizar_con_llm(review_text, customer_name, max_retries=3):
    """Llama a OpenRouter para analizar una reseña"""
    
    prompt = crear_prompt_analisis(review_text, customer_name)
    
    payload = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": "Eres un experto en análisis de reseñas de viajes. Respondes SOLO con JSON válido."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,  # Baja temperatura para respuestas consistentes
        "max_tokens": 500
    }
    
    for intento in range(max_retries):
        try:
            response = requests.post(
                OPENROUTER_API_URL,
                headers=headers,
                json=payload,
                timeout=30
            )
            
            if response.status_code == 200:
                result = response.json()
                content = result['choices'][0]['message']['content']
                
                # Intentar parsear el JSON
                # Limpiar el contenido si viene con markdown
                if "```json" in content:
                    content = content.split("```json")[1].split("```")[0]
                elif "```" in content:
                    content = content.split("```")[1].split("```")[0]
                
                parsed = json.loads(content.strip())
                return parsed
            
            elif response.status_code == 429:  # Rate limit
                time.sleep(2 ** intento)  # Exponential backoff
                continue
            
            else:
                print(f"Error API: {response.status_code} - {response.text}")
                return None
                
        except json.JSONDecodeError as e:
            print(f"Error parseando JSON: {e}")
            return None
        except Exception as e:
            print(f"Error en petición: {e}")
            if intento < max_retries - 1:
                time.sleep(1)
    
    return None

In [ ]:
def procesar_reseñas_batch(df, batch_size=10, start_index=0):
    """Procesa las reseñas en lotes para evitar límites de rate"""
    
    # Filtrar solo reseñas no analizadas con texto
    df_pendientes = df[
        (df['analyzed'] == False) & 
        (df['review_text'].notna()) & 
        (df['review_text'] != '')
    ].iloc[start_index:]
    
    print(f"Reseñas pendientes de analizar: {len(df_pendientes)}")
    
    resultados = []
    errores = []
    
    # Procesar en lotes
    for i in tqdm(range(0, len(df_pendientes), batch_size), desc="Procesando lotes"):
        batch = df_pendientes.iloc[i:i+batch_size]
        
        for idx, row in batch.iterrows():
            # Analizar reseña
            resultado = analizar_con_llm(
                row['review_text'], 
                row['customer_name']
            )
            
            if resultado:
                resultado['index'] = idx
                resultados.append(resultado)
            else:
                errores.append({
                    'index': idx,
                    'review_id': row['review_id'],
                    'error': 'No se pudo analizar'
                })
            
            # Pequeña pausa entre peticiones
            time.sleep(0.5)
        
        # Pausa más larga entre lotes
        print(f"Lote completado. Esperando antes del siguiente...")
        time.sleep(2)
    
    return resultados, errores

In [ ]:
def guardar_progreso(df, filename_base='trustpilot_analyzed'):
    """Guarda el progreso del análisis"""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{filename_base}_{timestamp}.csv"
    
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Progreso guardado en: {filename}")
    
    # También guardar un backup del último estado
    df.to_csv(f"{filename_base}_latest.csv", index=False, encoding='utf-8-sig')
    
    return filename

In [ ]:
def analizar_trustpilot_reviews(csv_path, batch_size=10, max_reviews=None):
    """Función principal para analizar las reseñas"""
    
    # Cargar datos
    print("📂 Cargando datos...")
    df = pd.read_csv(csv_path, encoding='utf-8-sig')
    
    # Limitar número de reseñas si se especifica
    if max_reviews:
        df = df.head(max_reviews)
    
    print(f"📊 Total de reseñas a procesar: {len(df)}")
    
    # Verificar API key
    if not OPENROUTER_API_KEY or OPENROUTER_API_KEY == "tu-api-key-aqui":
        print("❌ Error: Configura tu API key de OpenRouter")
        return None
    
    # Procesar reseñas
    print("\n🤖 Iniciando análisis con LLM...")
    resultados, errores = procesar_reseñas_batch(df, batch_size)
    
    # Actualizar DataFrame
    print(f"\n✅ Análisis completado: {len(resultados)} reseñas")
    print(f"❌ Errores: {len(errores)} reseñas")
    
    df_actualizado = actualizar_dataframe(df, resultados)
    
    # Guardar resultados
    filename = guardar_progreso(df_actualizado)
    
    # Mostrar estadísticas
    print("\n📈 Estadísticas del análisis:")
    print(f"- Idiomas detectados: {df_actualizado['language'].value_counts().to_dict()}")
    print(f"- Sentimientos: {df_actualizado['sentiment'].value_counts().to_dict()}")
    print(f"- Tipos de turista: {df_actualizado['tourist_type'].value_counts().to_dict()}")
    
    return df_actualizado, errores

In [ ]:
# Configurar parámetros
CSV_PATH = "trustpilot_travel_reviews_20241201_120000.csv"  # Ajusta el nombre
BATCH_SIZE = 10  # Reseñas por lote
MAX_REVIEWS = 100  # None para procesar todas

# Ejecutar análisis
df_analizado, errores = analizar_trustpilot_reviews(
    csv_path=CSV_PATH,
    batch_size=BATCH_SIZE,
    max_reviews=MAX_REVIEWS
)

In [ ]:
'''
# Crear visualizaciones básicas
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo
plt.style.use('seaborn-v0_8')
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Distribución de sentimientos
df_analizado['sentiment'].value_counts().plot(kind='bar', ax=axes[0,0])
axes[0,0].set_title('Distribución de Sentimientos')

# 2. Top 10 temas principales
df_analizado['main_topic'].value_counts().head(10).plot(kind='barh', ax=axes[0,1])
axes[0,1].set_title('Top 10 Temas Principales')

# 3. Tipos de turista
df_analizado['tourist_type'].value_counts().plot(kind='pie', ax=axes[1,0])
axes[1,0].set_title('Tipos de Turista')

# 4. Emociones detectadas
df_analizado['emotion'].value_counts().plot(kind='bar', ax=axes[1,1])
axes[1,1].set_title('Emociones Detectadas')

plt.tight_layout()
plt.show()
'''